In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Bidirectional, LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pandas as pd

import numpy as np
import json

In [ ]:
#cols = ['Bidirectional', 'Nr_layers',  'Layer_size', 'Regularization', 'Dropout', 'Loss', 'Accuracy']
#df = pd.DataFrame(columns = cols) #we use it to save grid results

modelname = 'Model1'

df = pd.read_csv('/content/drive/MyDrive/DATA/Original/'+modelname +"/"+"Results_grid.csv")

df = df.drop(columns=['Unnamed: 0'])

print(df)

     Bidirectional  Nr_layers  Layer_size  ...  Dropout      Loss  Accuracy
0            False          1          16  ...      0.0  0.399955  0.772857
1            False          1          16  ...      0.2  0.399516  0.772093
2            False          1          16  ...      0.4  0.399671  0.773795
3            False          1          16  ...      0.6  0.403331  0.773135
4            False          1          16  ...      0.0  0.405060  0.773968
..             ...        ...         ...  ...      ...       ...       ...
166           True          2          32  ...      0.4  0.408852  0.772822
167           True          2          32  ...      0.6  0.414702  0.770738
168           True          2          32  ...      0.0  0.414259  0.772891
169           True          2          32  ...      0.2  0.422095  0.772787
170           True          2          32  ...      0.4  0.428774  0.774559

[171 rows x 7 columns]


In [ ]:
def import_log(filepath):
  df = pd.read_csv(filepath)
  return(df.values.tolist())

def remove_nan(lists):
  newlists = []
  for tr in lists:
    newlists.append([int(x) for x in tr if str(x) != 'nan'])
  return(newlists)

def number_to_one_hot_X(X, dict_size): #if we want 
  newX = []
  for example in X:
    new_ex = []
    for i in range(len(example)):
      onehot = [0]*dict_size #changed
      if example[i] != 0:
        onehot[example[i] - 1] = 1 #-1 because begin counting at 0
      new_ex.append(onehot)
    newX.append(new_ex)
  return(np.array(newX))

def create_XY_prefix(log, mappingsize, prefixlen):
  X = []
  Y = []
  for i in range(0, len(log)):
    for k in range(1, len(log[i])):
      X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
      y = [0] *(mappingsize)
      y[int(log[i][k])-1] = 1
      Y.append(y)        
  X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
  X = number_to_one_hot_X(X, mappingsize)
  return(np.array(X), np.array(Y))



In [ ]:
def get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = Sequential()
  model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
  if bidirec == False:   
    model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2)))
      model.add(Dropout(dropout))
  else:
    model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars))))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2))))
      model.add(Dropout(dropout))
  model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
  opt = Adam(learning_rate=0.005)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics='accuracy')
  return model


def do_one_experiment(X_train, y_train, X_test, y_test,batch_size, maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2)
  model.summary()
  early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
  #train_model
  history = model.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping, lr_reducer], batch_size=batch_size, epochs=600, verbose=2)
  #test model on test set
  loss, accuracy = model.evaluate(X_test, y_test)
  return model, loss, accuracy

def do_grid_search(df, modelname, full_prefix, opt_prefixlen): #add df just a way to ontinue training in seperate runs
  path = '/content/drive/MyDrive/DATA/Original/'+modelname +"/"

  full_log = remove_nan(import_log(path+"Log_"+modelname+".csv")) #import full event log

  #if we want to use the full prefix each time or not
  if full_prefix == True:
    prefixlen=len(max(full_log,key=len)) -1
    print("prefix length:", prefixlen)
  else:
    prefixlen=opt_prefixlen

  #reload mapping
  mappingfilename = path+"Mapping_"+modelname+".txt"
  with open(mappingfilename) as f:
    mapping = json.loads(f.read())
  #split into train and test traces, the log is randomly played out, so no need to mix
  train_log = full_log[0: int(len(full_log) * 0.8)]
  test_log = full_log[int(len(full_log) * 0.8):-1]

  #convert into prefix + target, both one_hot_encoded
  X_train, y_train = create_XY_prefix(train_log, len(mapping), prefixlen)
  X_test, y_test = create_XY_prefix(test_log, len(mapping), prefixlen)
  
 

  grid_is_bidirectional = [False, True]
  grid_nr_layers = [1, 2]
  grid_layer_size = [16, 32, 64]
  grid_reg = [0.0, 0.0001, 0.001, 0.01]
  grid_dropout = [0.0 ,0.2, 0.4, 0.6]

  batch_size = 128

  i = 0 #keep track of index in grid

  for is_bidirectional in grid_is_bidirectional:
    for nr_layers in grid_nr_layers:
      for layer_size in grid_layer_size:
        for reg in grid_reg:
          for dropout in grid_dropout:
            if (i in df.index) == True:
              print(i, 'This setting already in dataframe:', is_bidirectional,nr_layers,layer_size,reg,dropout)
            else:
              model, loss, accuracy = do_one_experiment(X_train, y_train, X_test, y_test,batch_size, maxlen=prefixlen, num_chars=len(mapping), bidirec=is_bidirectional, n_layers=nr_layers, lstmsize=layer_size, dropout=dropout, l1=reg, l2=reg)
              new_row = {'Bidirectional':is_bidirectional,'Nr_layers':nr_layers,'Layer_size':layer_size, 'Regularization':reg,'Dropout':dropout,'Loss':loss,'Accuracy':accuracy}
              df = df.append(new_row, ignore_index=True)
              print(new_row)
              df.to_csv(path + 'Results_grid.csv')
              modelfilename = path + 'Hyper_Models/Model_Bidirec'+ str(is_bidirectional) + '_NL'+str(nr_layers)+'_LS'+str(layer_size)+'_R'+str(reg).replace('.', '')  +'_D'+str(dropout).replace('0.', '')
              model.save(modelfilename)
            i = i+1

In [ ]:
do_grid_search(df, 'Model1', full_prefix = False, opt_prefixlen = 6)

0 This setting already in dataframe: False 1 16 0.0 0.0
1 This setting already in dataframe: False 1 16 0.0 0.2
2 This setting already in dataframe: False 1 16 0.0 0.4
3 This setting already in dataframe: False 1 16 0.0 0.6
4 This setting already in dataframe: False 1 16 0.0001 0.0
5 This setting already in dataframe: False 1 16 0.0001 0.2
6 This setting already in dataframe: False 1 16 0.0001 0.4
7 This setting already in dataframe: False 1 16 0.0001 0.6
8 This setting already in dataframe: False 1 16 0.001 0.0
9 This setting already in dataframe: False 1 16 0.001 0.2
10 This setting already in dataframe: False 1 16 0.001 0.4
11 This setting already in dataframe: False 1 16 0.001 0.6
12 This setting already in dataframe: False 1 16 0.01 0.0
13 This setting already in dataframe: False 1 16 0.01 0.2
14 This setting already in dataframe: False 1 16 0.01 0.4
15 This setting already in dataframe: False 1 16 0.01 0.6
16 This setting already in dataframe: False 1 32 0.0 0.0
17 This setting a

INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS32_R0001_D6/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS32_R0001_D6/assets


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 6, 64)            11776     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 6, 64)             0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 13)                845       
                                                                 
Total params: 37,453
Trainable params: 37,453
Non-trai

INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS32_R001_D0/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS64_R001_D4/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS64_R001_D4/assets


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_40 (Bidirecti  (None, 6, 128)           39936     
 onal)                                                           
                                                                 
 dropout_40 (Dropout)        (None, 6, 128)            0         
                                                                 
 bidirectional_41 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_41 (Dropout)        (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 13)                1677      
                                                                 
Total params: 140,429
Trainable params: 140,429
Non-t

INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS64_R001_D6/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/DATA/Original/Model1/Hyper_Models/Model_BidirecTrue_NL2_LS64_R001_D6/assets
